# **Fourier Transform and planewave expansion**

<i class="fa fa-home fa-2x"></i><a href="./index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/LyKex/quantum-mechanics/blob/master/notebook/FFT_and_planewaves.ipynb (**needs update**)

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
    Planewave expansion is perhaps the most commonly adopted wavefunction representation when solving Khon-Sham equation. The possiblity to use fast Fourier transform (FFT) to determine the compostion of the planewaves make this strategy numerically efficient. This notebook shows interactively how discrete Fourier series can represent an objective function with limited amout of components.
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand why planewave expansion is perferred for numerical solution of Kohn-Sham equation.</li>
        <li> Understand the same nature of Fourier series and planewave basis. </li>
        <li> Learn how to decompose a function by FFT algorithm.</li>
        <li> Examine the reconstructed results from expansions with different objective functions. </li>
    </ol>

</details>

## **Background theory**

<p style="text-align: justify;font-size:15px">
    Density functional theory (DFT) calculation is largely based on solving Kohn-Sham equation self-consistently. The wavefunction of the system can be presented in multiple ways. For materials system with periodic boundary condition, planewave basis is frequently chosen due to its periodic nature and numerical efficiency. This strategy is adopted in many popular DFT packages including Quantum ESPRESSO and VASP.
</p>

<details close>
<summary style="font-size: 20px">Khon-Sham equation</summary>
<p style="text-align: justify;font-size:15px">
    The Kohn-Sham equation is
</p>

$-\frac{1}{2} \nabla^{2} \phi_{i}(\mathbf{r})+V_{\text {tot }}(\mathbf{r}) \phi_{i}(\mathbf{r})=\varepsilon_{i} \phi_{i}(\mathbf{r})$  (1)
  
<p style="text-align: justify;font-size:15px">
    where $V_{\text {tot}}$ is the total potential which is the sum of nuclear potential, $V_n$, electron Hatree potential, $V_H$ and exchange correlation potential, $V_{xc}$. The latter two are both determined by electron desnity, for example, the Poisson's equation gives the $V_H$ by
</p>

$\nabla^{2} V_H(\mathbf{r}) = -4 \pi n(r) $  (2)
  
<p style="text-align: justify;font-size:15px">
     Thus, to obtain a self-consistent solution in a typical DFT calculation, the trial charge density will be iteratively examined until $V_{\text {tot}}$ and $\phi_{i}(\mathbf{r})$ is converged in equation 1. However, this process can computationally expensive. Assuming setting up an  grid of $N\times N \times N$ equally spaced points, the wavefunction would have ~$N^3$ elements while the Hamiltonian contains ~$N^6$. Solving the eigenvalue of equation 1 would be quite difficult if N is large enough to have meaningful result.
</p>
</details>

<details close>
<summary style="font-size: 20px">Planewave expansion</summary>
<p style="text-align: justify;font-size:15px">
    Planewave is named after the fact that it has the wavefront is identical everywhere on the plane perpendicular to the propagation direction. It has the general form as
</p>

$ P(x) = P_0 e^{iqx}$  (3)

<p style="text-align: justify;font-size:15px">
    The planewave representation, i.e. to use a planewave basis set to represent the wavefunction of the whole system is 
</p>

$\phi_i(\mathbf{r}) = \sum_{\mathbf{G}} c_i(\mathbf G) e^{(i\mathbf{Gr})}$  (4)

<p style="text-align: justify;font-size:15px">
    Planewave expansions is desired for multiple reasons. Foremost, it is mathmatically a Fourier series which means FFT can be used to efficiently find the wave coefficients $c_n$. So the multiplication of $V_tot$ and $\phi_i$ can be done in real space and convert to reciprocal space by FFT. Second, under periodic boundary conditions, it naturally appers afer Bloch theorem.
</p>
    
<p style="text-align: justify;font-size:15px">
    It is important to point out that planewave expansion can only be effcient if we can reduce the number of elements in the Hamiltonian and $\phi_i$ since the eigenfuction still needs to be solved.
</p>
</details>

<details close>
<summary style="font-size: 20px">Fourier series and Fast fourier transform</summary>
<p style="text-align: justify;font-size:15px">
    Fouier series is a combination of cosine and sine functions to represent a smooth function defined on a certain range. For a one dimentional function $f(x)$ defined on [$-\pi$, $-\pi$], the Fourier series is given by   
</p>
$ f(x) = \frac {A_0} {2} + \sum_{k=1}^{\infty}(A_k \cos kx + B_k \sin kx)$  (5)

<p style="text-align: justify;font-size:15px">
    where the coeffcients are defined as
</p>
$ A_k = \frac {1}{\pi} \int_{-\pi}^{\pi} f(x) \cos(kx)dx = \frac {1} {\lVert\cos (kx)\rVert ^2} \langle f(x), \cos(kx)\rangle$  and <br/>
$ B_k = \frac {1}{\pi} \int_{-\pi}^{\pi} f(x) \sin(kx)dx = \frac {1} {\lVert\sin (kx)\rVert ^2} \langle f(x), \sin(kx)\rangle$
    
<p style="text-align: justify;font-size:15px">
   The definition of Fourier coefficient can be interpreted as the projection of a function onto a unit basis vector of cosine or since. Thus, the Fourier series is essentially the representation of a function using an infinite basis set. 
</p>
    
<p style="text-align: justify;font-size:15px">
     In complex space, Fourier shares the form as the planewave expansions.
</p>    

$ f(x)=\sum_{-\infty}^{\infty}C_k e^{ikx} = \sum_{-\infty}^{\infty}(\alpha_k + i\beta_k)(\cos kx + i\sin kx) $  (6)

<p style="text-align: justify;font-size:15px">
     Notice for both real and complex cases, the basis set is composed of trigonometry functions with increasing frequency. Since all the frequency is considered, the representation is precise and strictly equal. However, it is impractical to consider infinite amount of basis vector in numerical computation, so Discrete Fourier Series is often used to have a accurate enough representation, which is given by
</p>

$ \hat{f_k}= \sum_{n=0}^{N-1} f_n e^{-ikn/N} \quad k=0,1,...,N-1$  (7)
  
<p style="text-align: justify;font-size:15px">
   where $f_k$ is the k-th sampling of the objective functions and $\hat{f_k}$ is the Fourier coeffcients, with which the original function can be constructed by inverse Fourier transform 
</p>
$ f_k = \frac 1 N \sum_{n=0}^{N-1} \hat{f_n} e^{ikn/N}$  (8)

    
<p style="text-align: justify;font-size:15px">
   We have hinted in the previous section that Fourier transform is fast so that planewave expansion has advatage in computation. Yet, the Discrete Fourier Transform actually requires $O(N^2)$ time to compute which is clear if we express equation 6 in matrix form.
</p>
    
$\begin{pmatrix}\hat{f_0} \\\hat{f_1} \\\hat{f_2} \\ \vdots \\\hat{f_N} \end{pmatrix} = 
\begin{pmatrix}
1 & 1 & 1 &\dots &1 \\
1 & w_N & w_N^2 & \dots & w_N^{N-1}\\
1 & w_N^2 & w_N^4 & \dots & w_N^{2(N-1)}\\
\vdots & & & & \vdots \\
1 & w_N^{N-1} & w_N^{2(N-1)} & \dots & w_N^{(N-1)^2}
\end{pmatrix} \cdot 
\begin{pmatrix} f_0 \\ f_1 \\ f_2 \\ \vdots \\ f_{N-1} \end{pmatrix}
$
    
 <p style="text-align: justify;font-size:15px">
   
    Fast Fourier transform (FFT) is an algorithm to find $\hat{f_k}$ with only $O(N\lg N)$ time which means when it can scale up.  One of the earliest and most common FFT is Cooley-Tukey  method [<a href="https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/home.html">Math. Comp. 19, 297–301 (1965)</a>]. They use a divide and conquer algorithm by first breaking down the matrix multiplication into two smaller parts and then recursively apply the procedure. 
</p>
</details>

## **Tasks and exercises**

<ol style="text-align: justify;font-size:15px">
    <li> Why cosine and since functions in complex Fourier series can be used as basis set? Which requirement should be fullfillled?
    <details style="color: blue">
    <summary>Hints</summary>
        To represent wavefunctions, basis set has to be composed of orthoganal vectors. Thus, we have to prove that $\langle w_N^j, w_N^k \rangle =\langle w_N^k, w_N^j \rangle= 0 $ for integer $j \neq k$ where $w_N = e^{-2 \pi i /n}$. We can simply carry out the inner product  
     
        $ \langle w_N^j, w_N^k \rangle = \langle w_N^k, w_N^j \rangle = \int_{-\pi}^{\pi} e^{ijx} e^{-ikx}dx = \int_{-\pi}^{\pi} e^{i(j-k)x} dx = \frac 1 {i(j-k)} [e^{i(j-k)x}]_{-\pi}^{\pi} = \begin{cases} 0 & \text{if j $\neq$ k} \\ 2\pi & \text{if j = k}\end{cases}$
    </details>
    </li>
 
    <li> How the number of sampling is affecting the Fourier transform? Will a less continuous function requires more components to represent?
    <details style="color: blue">
    <summary>Hints</summary>
        Move the slide bar to try different number of Fourier components. Observe if the FFT interpolation fits the original function and if the integral of the square modulus is close to the convergence value. You can also change the objective function by the dropdown menu. Generally, more sampling yields more accurate representation while for rough functions, more Fourier components are needed to reach the same level of precision.
    </details>
    </li>
    
    <li> How can we reduce the number of planewaves needed without sacrificing the accuracy of the representation?
    <details style="color: blue">
    <summary>Hints</summary>
        Wavefunctions has the strongest oscillation near nucleus, and large number of planewaves are needed for rough functions. Fortunately, core electrons are less important for bonding so we can simplify the problem and obtain a much smoother wavefunction by excluding the core electrons. To learn more about this idea, please check our <a href="./pseudopotentials.ipynb" style="font-size: 15px">notebook on pseudopotential</a>. In general, the combination of pseudopotential and planewave expansions enables fast and reliable calculation on practical system scales.
    </details>
    </li>
    
    <li> In DFT calculation, how can we control the number of planewaves used in the basis set?
    <details style="color: blue">
    <summary>Hints</summary>
        The kinetic energy of planewave is given by $\frac {\hbar^2}{2m} \lvert \mathbf k + \mathbf G \rvert^2 $, where $\mathbf k$ is the space vector in the first Brilliouin zone. By setting a cutoff energy, we can limit the size of planewave basis. The value of the cutoff is subject to the specific system under investigation and convergence tests are normally required. To have a suggestion on the cutoff value based on the choice of pseudopotential, you can consult <a href="https://www.materialscloud.org/discover/sssp/table/precision" style="font-size: 15px">standard solid-state pseudopotentials library</a> in Materials Cloud project. 
    </details>
    </li>
</ol>

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

In [16]:
import ipywidgets as ipw
import numpy as np
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import scipy.fft as fft
%matplotlib widget
plt.rcParams['figure.autolayout'] = 'True' # turn on tight layout globally

In [17]:
# objective functions
def periodic_f(x):
    # smooth
    return np.exp(-((x-1)/0.15)**2) + 0.5 * np.exp(-((x-1.2)/0.1)**2) + 0.8 * np.exp(-((x-0.8)/0.1)**2)

def periodic_f2(x):
    # less smooth
    return np.exp(-((x-1)/0.05)**2) -0.5*np.exp(-((x-1)/0.15)**2) + 0.5 * np.exp(-((x-1.2)/0.1)**2) + 0.8 * np.exp(-((x-0.8)/0.1)**2)

In [18]:
# plot x range
x = np.linspace(0, 2, 201, endpoint=False)
x_range = 2

# number of fft samples
N_slider = ipw.IntSlider(description=r"$N_{\text{fft}}$", min=6, max=40, value=6, step=1, continuous_update=False)
# objective functions
func_dropdown=ipw.Dropdown(description="Function", options=[("Smooth", "periodic_f"), ("Less smooth", "periodic_f2")])
reset_button = ipw.Button(description='Reset', icon='redo', button_style='success')

In [19]:
# N_fft = 6
# x_fft = np.array([])
# y_fft = np.array([])
# y_resamp = np.array([])
# func = periodic_f

def compute_resampled(N_fft, x_range=2., function=periodic_f):
    """ Compute FFT series with given number of sampling and objective functions. """
    # Pick an even number to have zero
    x_fft = np.linspace(0, x_range, N_fft+1, endpoint=False)# remove last point as it's the same as the first one by PBC
    y_fft = function(x_fft)
    # Fourier resampling
    renormalization = len(x)/(len(y_fft))
    y_resamp = fft.irfft(fft.rfft(y_fft), len(x)) * renormalization
    
    return x_fft, y_fft, y_resamp

def get_integral_resampled(N_fft, x_range=2., function=periodic_f):
    """ Compute the integral of the square modulus of the function. """ 
    x_fft, y_fft, _ = compute_resampled(N_fft, x_range, function=function)
    return (y_fft**2).sum() * (x_fft[1] - x_fft[0])

def plot_reconstruct(y_fft):
    """ Plot Fourier expansions """
    ax2.clear()

    coeffs = fft.rfft(y_fft)
    N_rfft = 0 # number of fft expansions
    for coeff, freq_int in list(zip(coeffs, range(len(coeffs)))):
        freq = 2 * np.pi * freq_int / x_range
        norm = 1 / (len(y_fft)) * 2
        if freq_int == 0:
            # The zero-frequency does not have a factor 2 because it's not a cosine
            # summing the two complex conjugates, but just a constant
            norm /= 2
        this_frequency_contrib = ( coeff.real * np.cos(freq * x) - coeff.imag * np.sin(freq * x) ) * norm

        ax2.plot(x, this_frequency_contrib + N_rfft) # plot components with vertical shift for visibility
        # ax2.plot(x, this_frequency_contrib) # no shift
        N_rfft += 1

    ax2.axes.yaxis.set_ticks([]) # remove y ticks
    ax2.set_title('Expansion Components')



CONVERGE_SMOOTH = get_integral_resampled(N_fft=200, function=periodic_f)
CONVERGE_ROUGH = get_integral_resampled(N_fft=200, function=periodic_f2)
def plot_integral(func_name, func):
    """ plot sum of the square modulus (integral) """
    ax3.clear()
    converged_integral = CONVERGE_SMOOTH if func_name == "periodic_f" else CONVERGE_ROUGH
    ax3.axhline(converged_integral, color='tab:red')
    integrals = []
    for N in range(6, 41):
        integrals.append((N, get_integral_resampled(N, function=func)))
    integrals_x, integrals_y = np.array(integrals).T
    
    ax3.plot(integrals_x, integrals_y, 'o--', alpha=0.8)
    ax3.plot(integrals_x[N_fft-6], integrals_y[N_fft-6],'ro', markersize=11, label='current sampling')
    ax3.set_xlabel('number of components')
    ax3.set_ylabel("Integral of square modulus")
    ax3.set_title("Convergence of FFT")
    ax3.set_xlim(6,40)
    ax3.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax3.legend(loc='best')

def plot_sampling(func, x_fft, y_fft, y_resamp):
    ax1.clear()
    ax1.set_title('FFT sampling')

    x_fft, y_fft, y_resamp = compute_resampled(N_slider.value, function=func)
    ax1.plot(x, func(x), 'k-', label='objective function')
    ax1.plot(x_fft, y_fft, 'o', label='sampling points')
    ax1.fill_between(x, y_resamp, 0,ec='red', fc='yellow', label='FFT interpolation')
    ax1.legend(loc='best')
    ax1.set_ylim(-0.3,1.25)

def on_plot_click(event):
    """handle mouse click event on expansion component plot"""
    # line = event.artist
    # xdata = line.get_xdata()
    # ydata = line.get_ydata()
    if event.inaxes != ax2:
        return
    for i in range(len(ax2.lines)):
        ax2.lines[i].set_alpha(0.1)
        ax2.lines[i].set_linewidth(1.1)

    # get the id of the line2D object which is vertically closest to the mouse clicking position 
    id_line = min(enumerate(ax2.lines), key= lambda line: abs(np.mean(line[1].get_ydata())-event.ydata))[0]
    ax2.lines[id_line].set_alpha(1)
    ax2.lines[id_line].set_linewidth(2.0)

    plot_sampling(func, x_fft, y_fft, y_resamp)
    ax1.fill_between(ax2.lines[id_line].get_xdata(), ax2.lines[id_line].get_ydata()-id_line, 0, ec='tab:blue', fc='tab:green', alpha=0.5,label='component chosen')
    ax1.legend()

def plot_update(change):
    # get current widget value
    global N_fft, x_fft, y_fft, y_resamp, func
    N_fft = N_slider.value
    func = globals()[func_dropdown.value] # get the function by function name
    x_fft, y_fft, y_resamp = compute_resampled(N_fft, function=func)
    # update sampling plot
    plot_sampling(func, x_fft, y_fft, y_resamp)

    # update reconstruct plot
    plot_reconstruct(y_fft)

    # udpate square modulus plot
    plot_integral(func_dropdown.value, func)


N_slider.observe(plot_update, names='value', type='change')
func_dropdown.observe(plot_update, names='value', type='change')
reset_button.on_click(plot_update)

In [20]:
# define layout by gridspec
fig = plt.figure(constrained_layout=True, figsize=(10, 7.5))
gs = fig.add_gridspec(3,4)
ax1 = fig.add_subplot(gs[0:2,0:2])
ax2 = fig.add_subplot(gs[0:2,2:4])
ax3 = fig.add_subplot(gs[-1,:])

# interactive plot 2 line picking
cid = fig.canvas.mpl_connect('button_press_event', on_plot_click)

# show plots
plot_update(None)
plt.show()

# display widgets
box_layout = ipw.Layout(display='flex', align_items='stretch',width='80%', flex_flow='row')
box = ipw.Box(children=[reset_button, N_slider, func_dropdown], layout=box_layout)
display(box)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Box(children=(Button(button_style='success', description='Reset', icon='redo', style=ButtonStyle()), IntSlider…

<details>
    <summary style="font-size: 22px;"><b>Legend</b></summary>

<p style="text-align: justify;font-size:15px">
    The number of FFT components, $N_{\text{fft}}$ is set by the slider. Two objective functions can be chosen from the dropdown menu. The objective function, sampling points and the reconstructed function are shown in plot 1 on the top left. The real part (coisine functions) and the constant term of the Fourier series is shown in plot 2 on the top right. Note that the components are shifted vertically for better visibility. The norm integral of the Fourier series with different $N_{\text{fft}}$ is shown in plot at bottom where the current choice of sampling is labeled. The convergence value is obtained with 200 FFT components. 
   
</p>
</details>